In [72]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pymongo import MongoClient
from pprint import pprint
from helpers import zero_to_nan
import json

### Pull data from MongoDB

In [2]:
client = MongoClient()
db = client.get_database('philadelphia_home_prices')
col = db.get_collection('listing_data')

In [3]:
cursor = db['listing_data'].find({})

In [4]:
df = pd.DataFrame(list(cursor))

In [5]:
df.head(5)

,_id,Address,Locality,Region,Postal Code,Price,Days on Market,SQFT House,SQFT Lot,Full Baths,...,Build Year,Remodel Year,Neighborhood,Neighborhood median sales,Neighborhood Median DOM,Neighborhood Price SQFT,Association,Association Monthly,URL,crawl time
0,5fa7394e653eadd27d60c054,"1211 E Cheltenham Ave,",Philadelphia,PA,19124,100000.0,44,1110.0,1080,1,...,1950,2020,Summerdale,Not Found,Not Found,120,No,0,https://www.realtor.com/realestateandhomes-det...,2020-11-07 17:18:22.082
1,5fa73952653eadd27d60c055,"5254 N Howard St,",Philadelphia,PA,19120,161650.0,55,1230.0,1050,2,...,1925,2020,Olney,123000,67,113,No,0,https://www.realtor.com/realestateandhomes-det...,2020-11-07 17:18:26.690
2,5fa73959653eadd27d60c056,"3429 W Queen Ln,",Philadelphia,PA,19129,327000.0,36,1383.0,1381,2,...,1929,2020,East Falls,317500,56,220,No,0,https://www.realtor.com/realestateandhomes-det...,2020-11-07 17:18:33.404
3,5fa7395f653eadd27d60c057,"212 Carpenter St Unit C,",Philadelphia,PA,19147,415000.0,49,1440.0,785,1,...,1920,2020,Queen Village,550500,60,365,No,0,https://www.realtor.com/realestateandhomes-det...,2020-11-07 17:18:39.194
4,5fa73963653eadd27d60c058,"3340 Tyson Ave,",Philadelphia,PA,19149,185000.0,60,1099.0,1678,1,...,1950,2020,Mayfair,207500,74,160,No,0,https://www.realtor.com/realestateandhomes-det...,2020-11-07 17:18:43.820


In [6]:
df = df.drop(['_id','URL','crawl time'],1)

In [7]:
df = df.set_index('Address')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 1211 E Cheltenham Ave, to 7321 Montour St,
Data columns (total 24 columns):
Locality                     1283 non-null object
Region                       1283 non-null object
Postal Code                  1283 non-null object
Price                        1266 non-null float64
Days on Market               1309 non-null int64
SQFT House                   1212 non-null float64
SQFT Lot                     1309 non-null int64
Full Baths                   1309 non-null int64
Half Baths                   1309 non-null int64
Bedrooms                     1137 non-null float64
Garage                       1309 non-null object
Master Bath                  1309 non-null object
Cooling                      1309 non-null object
Heating                      1309 non-null object
Annual Tax                   1309 non-null object
House Type                   1309 non-null object
Build Year                   1309 non-null object
Remodel Year    

In [9]:
df.head()

,Locality,Region,Postal Code,Price,Days on Market,SQFT House,SQFT Lot,Full Baths,Half Baths,Bedrooms,...,Annual Tax,House Type,Build Year,Remodel Year,Neighborhood,Neighborhood median sales,Neighborhood Median DOM,Neighborhood Price SQFT,Association,Association Monthly
Address,,,,,,,,,,,,,,,,,,,,,
"1211 E Cheltenham Ave,",Philadelphia,PA,19124,100000.0,44,1110.0,1080,1,0,3.0,...,1257,"[ Interior Row/Townhouse, Multi family]",1950,2020,Summerdale,Not Found,Not Found,120,No,0
"5254 N Howard St,",Philadelphia,PA,19120,161650.0,55,1230.0,1050,2,0,3.0,...,1307,"[ Interior Row/Townhouse, Multi family]",1925,2020,Olney,123000,67,113,No,0
"3429 W Queen Ln,",Philadelphia,PA,19129,327000.0,36,1383.0,1381,2,0,3.0,...,3123,"[ Interior Row/Townhouse, Multi family]",1929,2020,East Falls,317500,56,220,No,0
"212 Carpenter St Unit C,",Philadelphia,PA,19147,415000.0,49,1440.0,785,1,1,3.0,...,5312,"[ Interior Row/Townhouse, Multi family]",1920,2020,Queen Village,550500,60,365,No,0
"3340 Tyson Ave,",Philadelphia,PA,19149,185000.0,60,1099.0,1678,1,0,3.0,...,2154,"[ Interior Row/Townhouse, Multi family]",1950,2020,Mayfair,207500,74,160,No,0


### Process columns

In [10]:
df.isnull().sum()

Locality                      26
Region                        26
Postal Code                   26
Price                         43
Days on Market                 0
SQFT House                    97
SQFT Lot                       0
Full Baths                     0
Half Baths                     0
Bedrooms                     172
Garage                         0
Master Bath                    0
Cooling                        0
Heating                        0
Annual Tax                     0
House Type                     0
Build Year                     0
Remodel Year                   0
Neighborhood                   0
Neighborhood median sales      0
Neighborhood Median DOM        0
Neighborhood Price SQFT        0
Association                    0
Association Monthly            0
dtype: int64

##### check price

In [11]:
#no free houses. change potential zeros to NaN
df.Price = df.Price.apply(zero_to_nan)

##### check days on market

In [12]:
#check if there is NaN
df['Days on Market'].isnull().sum()

0

##### check SQFT House

In [13]:
#zero sqft house is a lot
df['SQFT House'] = df['SQFT House'].apply(zero_to_nan)

##### check full baths

In [14]:
df['Full Baths'].value_counts()

1     650
2     321
0     231
3      95
4       7
6       2
5       2
17      1
Name: Full Baths, dtype: int64

In [15]:
df[df['Full Baths']>6] 

,Locality,Region,Postal Code,Price,Days on Market,SQFT House,SQFT Lot,Full Baths,Half Baths,Bedrooms,...,Annual Tax,House Type,Build Year,Remodel Year,Neighborhood,Neighborhood median sales,Neighborhood Median DOM,Neighborhood Price SQFT,Association,Association Monthly
Address,,,,,,,,,,,,,,,,,,,,,
"1520 N Sydenham St,",Philadelphia,PA,19121,1548400.0,0,5827.0,2120,17,0,18.0,...,15287,[ Multi family],2018,2020,North Central,238450,143,168,No,0


In [16]:
#NaN if full bath > 6
df['Full Baths'] = df['Full Baths'].apply(lambda x: np.nan if x> 6 else x)

##### check half baths

In [17]:
#check half baths
df['Half Baths'].value_counts()

0    868
1    420
2     21
Name: Half Baths, dtype: int64

##### check bedrooms

In [18]:
df.Bedrooms.value_counts()

3.0     648
2.0     204
4.0     142
0.0      60
1.0      37
5.0      32
6.0      10
7.0       2
18.0      1
8.0       1
Name: Bedrooms, dtype: int64

In [19]:
df[df['Bedrooms']>6]

,Locality,Region,Postal Code,Price,Days on Market,SQFT House,SQFT Lot,Full Baths,Half Baths,Bedrooms,...,Annual Tax,House Type,Build Year,Remodel Year,Neighborhood,Neighborhood median sales,Neighborhood Median DOM,Neighborhood Price SQFT,Association,Association Monthly
Address,,,,,,,,,,,,,,,,,,,,,
"23739 E Wister St,",Philadelphia,PA,19144,210000.0,32,5580.0,0,3.0,0,8.0,...,Not Found,[ Detached],1900,Not Updated,Not Found,250000,63,195,No,0
"1520 N Sydenham St,",Philadelphia,PA,19121,1548400.0,0,5827.0,2120,NaN,0,18.0,...,15287,[ Multi family],2018,2020,North Central,238450,143,168,No,0
"114 W Coulter St,",Philadelphia,PA,19144,560250.0,41,3419.0,6868,2.0,1,7.0,...,2923,"[ Detached, Single family]",1877,2020,Upper Northwest,318750,62,161,No,0
"114 W Coulter St,",Philadelphia,PA,19144,560250.0,41,3419.0,6868,2.0,1,7.0,...,2923,"[ Detached, Single family]",1877,2020,Upper Northwest,318750,62,161,No,0


In [20]:
#NaN if bedrooms >=18
df['Full Baths'] = df['Full Baths'].apply(lambda x: np.nan if x>= 18 else x)

##### check for garage

In [21]:
df.Garage.value_counts()

No      1046
[ 1]     229
[ 2]      28
[ 3]       3
[ 6]       2
[ 4]       1
Name: Garage, dtype: int64

In [22]:
#convert to numerical idicator of garage or garage spaces
def clean_garage(x):
    if type(x) == list:
        x = int(x[0])
    elif x == 'Yes':
        x = 1
    else:
        x = 0
    
    return x

In [23]:
df['Garage'] = df['Garage'].map(clean_garage)

In [24]:
df['Garage'].value_counts()

0    1046
1     229
2      28
3       3
6       2
4       1
Name: Garage, dtype: int64

##### check master bath

In [25]:
df['Master Bath'].value_counts()

No    1309
Name: Master Bath, dtype: int64

In [26]:
#unlikely there is not one mention of master bath. Drop master bath from consideration, check extractors.py 
df = df.drop(['Master Bath'],1)

#### check cooling and heating

In [27]:
#all options were some type of A/C or No A/C. Convert A/C options to 1 and no a/c to 0
df['Cooling'] = df['Cooling'].apply(lambda x: 0 if x=='No' else 1)

In [28]:
df.Heating.value_counts()

No                                                                     244
[Forced Air Heating, Yes]                                              243
[Forced Air Heating, Yes, Yes]                                         150
[Yes]                                                                   91
[Unknown]                                                               78
                                                                      ... 
[Radiator, Wall Unit Heating, Yes]                                       1
[Forced Air Heating, Heat Pump, Yes, Yes]                                1
[Forced Air Heating, Programmable Thermostat Heating, Yes, Unknown]      1
[Hot Water, Radiator, Yes, Unknown]                                      1
[Zoned Heating, Yes]                                                     1
Name: Heating, Length: 81, dtype: int64

In [29]:
#drop heating for now, but revist to increase accuracy
df = df.drop('Heating',1)

##### check annual tax

In [30]:
df['Annual Tax'].value_counts()

Not Found    128
946            5
1328           5
803            4
1888           4
            ... 
1001           1
3045           1
3281           1
3043           1
1002           1
Name: Annual Tax, Length: 871, dtype: int64

In [31]:
df['Annual Tax'] = df['Annual Tax'].apply(lambda x: np.nan if x == 'Not Found' else x)

##### check house type

In [32]:
df['House Type'].head(5)

Address
1211 E Cheltenham Ave,      [ Interior Row/Townhouse,  Multi family]
5254 N Howard St,           [ Interior Row/Townhouse,  Multi family]
3429 W Queen Ln,            [ Interior Row/Townhouse,  Multi family]
212 Carpenter St Unit C,    [ Interior Row/Townhouse,  Multi family]
3340 Tyson Ave,             [ Interior Row/Townhouse,  Multi family]
Name: House Type, dtype: object

In [33]:
def clean_house_type(x):
    x = ''.join(x)
    x = x.lower()
    x = x.strip()
    if ('apart' or 'unit') in x:
        x = "condo"
    elif ('townhouse') in x:
        x = 'townhouse'
    elif 'twin' in x:
        x = 'townhouse'
    elif ('single') in x:
        x = 'single family'
    elif 'land' in x:
        x = np.nan
    elif 'commercial' in x:
        x = np.nan
    elif 'multi' in x:
        x = 'townhouse'
    elif '' in x:
        x = np.nan
    return x

In [34]:
df['House Type'] = df['House Type'].apply(clean_house_type)

In [35]:
df['House Type'].value_counts()

townhouse        970
single family    117
condo             86
Name: House Type, dtype: int64

##### check build year

In [36]:
df['Build Year'].value_counts()

1925         208
1920         164
1950         120
Not Found     87
1900          55
            ... 
1913           1
1989           1
1974           1
1919           1
1927           1
Name: Build Year, Length: 98, dtype: int64

In [37]:
#drop missing build years
df['Build Year'] = df['Build Year'].apply(lambda x: np.nan if x == 'Not Found' else x)

##### check remodel year

In [38]:
df['Remodel Year'].value_counts()

2020           1192
Not Updated     117
Name: Remodel Year, dtype: int64

In [39]:
#drop remodel year as this data looks wrong
df = df.drop(['Remodel Year'],1)

##### Neighborhood

In [40]:
df['Neighborhood'].value_counts()

Not Found            150
North Delaware        40
Mayfair               40
Center City           39
Point Breeze          37
                    ... 
Sharswood              1
Wissahickon Hills      1
West Poplar            1
Fairhill               1
Walnut Hill            1
Name: Neighborhood, Length: 127, dtype: int64

##### Neighborhood Median Sales

In [41]:
df['Neighborhood median sales'].value_counts()

250000       209
Not Found     63
207500        45
185000        38
510000        38
            ... 
335000         1
85000          1
457500         1
275000         1
158000         1
Name: Neighborhood median sales, Length: 92, dtype: int64

In [42]:
df['Neighborhood median sales'] = df['Neighborhood median sales'].apply(lambda x: np.nan if x == 'Not Found' else x)

##### Neighborhood Median DOM

In [43]:
df['Neighborhood Median DOM']

Address
1211 E Cheltenham Ave,      Not Found
5254 N Howard St,                  67
3429 W Queen Ln,                   56
212 Carpenter St Unit C,           60
3340 Tyson Ave,                    74
                              ...    
21418 New St Unit B,               63
2200 Arch St #307,                 63
1319 S 20th St,                    69
200 Locust St Unit 22EN,           71
7321 Montour St,                   51
Name: Neighborhood Median DOM, Length: 1309, dtype: object

In [44]:
#remove not found numbers
df['Neighborhood Median DOM'].apply(lambda x: np.nan if x == 'Not Found' else x)

Address
1211 E Cheltenham Ave,       NaN
5254 N Howard St,           67.0
3429 W Queen Ln,            56.0
212 Carpenter St Unit C,    60.0
3340 Tyson Ave,             74.0
                            ... 
21418 New St Unit B,        63.0
2200 Arch St #307,          63.0
1319 S 20th St,             69.0
200 Locust St Unit 22EN,    71.0
7321 Montour St,            51.0
Name: Neighborhood Median DOM, Length: 1309, dtype: float64

##### Neighborhood price sqft

In [45]:
df['Neighborhood Price SQFT'].value_counts()

195    198
160     78
191     55
151     38
368     38
      ... 
89       1
156      1
342      1
87       1
246      1
Name: Neighborhood Price SQFT, Length: 91, dtype: int64

In [46]:
df['Neighborhood Price SQFT'] = df['Neighborhood Price SQFT'].apply(lambda x: np.nan if x == 'Not Found' else x)

##### check association Yes/No

In [47]:
df['Association'].value_counts()

No     1272
Yes      37
Name: Association, dtype: int64

##### check association prices

In [48]:
df['Association Monthly'].value_counts()

0       1185
150        4
235        3
145        3
100        3
        ... 
322        1
1105       1
281        1
265        1
302        1
Name: Association Monthly, Length: 86, dtype: int64

### check dataframe integrity

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 1211 E Cheltenham Ave, to 7321 Montour St,
Data columns (total 21 columns):
Locality                     1283 non-null object
Region                       1283 non-null object
Postal Code                  1283 non-null object
Price                        1266 non-null float64
Days on Market               1309 non-null int64
SQFT House                   1212 non-null float64
SQFT Lot                     1309 non-null int64
Full Baths                   1308 non-null float64
Half Baths                   1309 non-null int64
Bedrooms                     1137 non-null float64
Garage                       1309 non-null int64
Cooling                      1309 non-null int64
Annual Tax                   1181 non-null float64
House Type                   1173 non-null object
Build Year                   1222 non-null float64
Neighborhood                 1309 non-null object
Neighborhood median sales    1246 non-null float64
Neighborhood 

In [50]:
df = df.drop_duplicates()

In [51]:
df = df.dropna()

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809 entries, 5254 N Howard St, to 1407 E Upsal St,
Data columns (total 21 columns):
Locality                     809 non-null object
Region                       809 non-null object
Postal Code                  809 non-null object
Price                        809 non-null float64
Days on Market               809 non-null int64
SQFT House                   809 non-null float64
SQFT Lot                     809 non-null int64
Full Baths                   809 non-null float64
Half Baths                   809 non-null int64
Bedrooms                     809 non-null float64
Garage                       809 non-null int64
Cooling                      809 non-null int64
Annual Tax                   809 non-null float64
House Type                   809 non-null object
Build Year                   809 non-null float64
Neighborhood                 809 non-null object
Neighborhood median sales    809 non-null float64
Neighborhood Median DOM      809 non

### Make dummies for categorical features

In [55]:
dummies_house_type = pd.get_dummies(df['House Type'],drop_first=True)
df = pd.concat([df, dummies_house_type], axis='columns')

In [ ]:
dummies_neighborhood = pd.get_dummies(df['Neighborhood'],drop_first=True)
df = pd.concat([df, dummies_neighborhood], axis='columns')

In [59]:
dummies_association = pd.get_dummies(df['Association'],drop_first=True)
df = pd.concat([df, dummies_association], axis='columns')

In [62]:
dummies_postal_code = pd.get_dummies(df['Postal Code'], drop_first=True)
df = pd.concat([df, dummies_postal_code], axis='columns')

In [65]:
df = df.drop(['House Type', 'Neighborhood', 'Association', 'Postal Code'], 1)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 809 entries, 5254 N Howard St, to 1407 E Upsal St,
Data columns (total 65 columns):
Locality                     809 non-null object
Region                       809 non-null object
Price                        809 non-null float64
Days on Market               809 non-null int64
SQFT House                   809 non-null float64
SQFT Lot                     809 non-null int64
Full Baths                   809 non-null float64
Half Baths                   809 non-null int64
Bedrooms                     809 non-null float64
Garage                       809 non-null int64
Cooling                      809 non-null int64
Annual Tax                   809 non-null float64
Build Year                   809 non-null float64
Neighborhood median sales    809 non-null float64
Neighborhood Median DOM      809 non-null object
Neighborhood Price SQFT      809 non-null float64
Association Monthly          809 non-null int64
single family                809 non

### Export to MongoDB

In [70]:
df.reset_index(inplace=True)

In [74]:
prices_dict = df.to_json(orient="index")
parsed = json.loads(prices_dict)
list_of_dicts = [value for value in parsed.values()]

In [75]:
list_of_dicts

[{'Address': '5254 N Howard St,',
  'Locality': 'Philadelphia',
  'Region': 'PA',
  'Price': 161650.0,
  'Days on Market': 55,
  'SQFT House': 1230.0,
  'SQFT Lot': 1050,
  'Full Baths': 2.0,
  'Half Baths': 0,
  'Bedrooms': 3.0,
  'Garage': 0,
  'Cooling': 1,
  'Annual Tax': 1307.0,
  'Build Year': 1925.0,
  'Neighborhood median sales': 123000.0,
  'Neighborhood Median DOM': 67,
  'Neighborhood Price SQFT': 113.0,
  'Association Monthly': 0,
  'single family': 0,
  'townhouse': 1,
  'Yes': 0,
  '19103': 0,
  '19104': 0,
  '19106': 0,
  '19107': 0,
  '19111': 0,
  '19114': 0,
  '19115': 0,
  '19116': 0,
  '19118': 0,
  '19119': 0,
  '19120': 1,
  '19121': 0,
  '19122': 0,
  '19123': 0,
  '19124': 0,
  '19125': 0,
  '19126': 0,
  '19127': 0,
  '19128': 0,
  '19129': 0,
  '19130': 0,
  '19131': 0,
  '19132': 0,
  '19133': 0,
  '19134': 0,
  '19135': 0,
  '19136': 0,
  '19137': 0,
  '19138': 0,
  '19139': 0,
  '19140': 0,
  '19141': 0,
  '19142': 0,
  '19143': 0,
  '19144': 0,
  '19145': 

#### Put in MongoDB

In [78]:
db.create_collection('listing_data_clean')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'philadelphia_home_prices'), 'listing_data_clean')

In [79]:
col = db.get_collection('listing_data_clean')

In [81]:
col.insert_many(list_of_dicts)

In [83]:
col.count_documents({})

809